In [1]:
import onnxruntime as ort
from optimum.onnxruntime import ORTStableDiffusionImg2ImgPipeline
from diffusers.utils import load_image
import torchvision.transforms as transforms

print(ort.get_device())

model_id ="stabilityai/sd-turbo"
pipeline = ORTStableDiffusionImg2ImgPipeline.from_pretrained(
    model_id,
    dtype="fp16",            # FP16은 GPU provider에서만
    use_io_binding=True,
    export=True,
    provider="CUDAExecutionProvider"  # 꼭 지정
)

c:\Users\Taeyong_Sim\anaconda3\envs\stable_diffusion\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Taeyong_Sim\anaconda3\envs\stable_diffusion\lib\site-packages\torch\onnx\_internal\registration.py:167: OnnxExporterWarning: Symbolic function 'aten::scaled_dot_product_attention' already registered for opset 14. Replacing the existing function with new function. This is unexpected. Please report it on https://github.com/pytorch/pytorch/issues.
  warnings.warn(


GPU


Keyword arguments {'subfolder': '', 'trust_remote_code': False} are not expected by StableDiffusionPipeline and will be ignored.
Loading pipeline components...: 100%|██████████| 5/5 [00:00<00:00,  5.52it/s]
You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .
c:\Users\Taeyong_Sim\anaconda3\envs\stable_diffusion\lib\site-packages\transformers\models\clip\modeling_clip.py:236: TracerWarning: Converti

In [2]:
import io, os
from typing import Optional
from flask import Flask, request, send_file, jsonify
from flask_cors import CORS
from PIL import Image
import numpy as np

In [3]:
import base64

# style transfer ONNX 경로 (이미 있다고 가정한 변수)
%cd ./
style_transfer_model_path = "./models/AdaIN.onnx"

# AdaIN ONNX 세션 생성
adain_session = ort.InferenceSession(
    style_transfer_model_path,
    providers=["CUDAExecutionProvider", "CPUExecutionProvider"]
)

# 서버에 있는 스타일 이미지 리스트 (원하는 걸로 바꿔 넣기)
STYLE_IMAGE_DIR = "./styles"
STYLE_IMAGE_FILES = [
    os.path.join(STYLE_IMAGE_DIR, "style1.jpg"),
    os.path.join(STYLE_IMAGE_DIR, "style2.jpg"),
    os.path.join(STYLE_IMAGE_DIR, "style3.jpg"),
    os.path.join(STYLE_IMAGE_DIR, "style4.jpg"),
]

STYLE_IMAGES = [Image.open(p).convert("RGB") for p in STYLE_IMAGE_FILES]


c:\Users\Taeyong_Sim\Desktop\School\3-2\AR, MR, VR\GitHub\ARVRMR2025_Project\Server


In [4]:
def img_to_nchw(img: Image.Image, size: int = 512) -> np.ndarray:
    img = img.resize((size, size))
    arr = np.array(img).astype("float32") / 255.0   # HWC, 0~1
    arr = np.transpose(arr, (2, 0, 1))             # CHW
    arr = np.expand_dims(arr, 0)                   # 1CHW
    return arr

def nchw_to_img(arr: np.ndarray) -> Image.Image:
    # arr: (1,3,H,W) or (3,H,W)
    if arr.ndim == 4:
        arr = arr[0]
    arr = np.clip(np.transpose(arr, (1, 2, 0)), 0.0, 1.0)  # HWC
    arr_uint8 = (arr * 255).astype("uint8")
    return Image.fromarray(arr_uint8)

def run_adain(content_img: Image.Image, style_img: Image.Image) -> Image.Image:
    c = img_to_nchw(content_img, size=512)
    s = img_to_nchw(style_img, size=512)
    out = adain_session.run(None, {"content": c, "style": s})[0]  # (1,3,512,512)
    return nchw_to_img(out)

def pil_to_base64(img: Image.Image) -> str:
    buf = io.BytesIO()
    img.save(buf, format="PNG")
    buf.seek(0)
    return base64.b64encode(buf.getvalue()).decode("utf-8")


In [27]:
# -----------------------
# 2) Flask 앱
# -----------------------
app = Flask(__name__)
CORS(app)  # WebGL/브라우저에서 접근할 경우를 대비

def to_rgb_image(file_storage) -> Image.Image:
    data = file_storage.read()
    img = Image.open(io.BytesIO(data)).convert("RGB")
    return img

@app.route("/health", methods=["GET"])
def health():
    return jsonify({"status": "ok", "provider": "CUDAExecutionProvider"})

@app.route("/infer", methods=["POST"])
def infer_combo():
    """
    요청:
      - form-data:
        - image: PNG 파일 (Unity에서 보낸 원본)
        - static_prompt (옵션)
        - dynamic_prompt (옵션)
        - strength, guidance_scale, steps, seed, width, height (SD용)
        - use_sd: "true" / "false"
        - num_styles: "4"  (몇 장의 AdaIN 스타일을 만들지)
    응답(JSON):
    {
      "sd": "<base64 png or null>",
      "styles": [
        {"id": "0", "image": "<base64 png>"},
        ...
      ]
    }
    """
    if "image" not in request.files:
        return jsonify({"error": "no image file"}), 400

    img = to_rgb_image(request.files["image"])   # 기존에 쓰던 함수 재사용

    # --- 옵션 파라미터들 ---
    use_sd = request.form.get("use_sd", "false").lower() == "true"
    static_prompt = request.form.get("static_prompt", "")
    dynamic_prompt = request.form.get("dynamic_prompt", "")
    prompt = (static_prompt + ", " + dynamic_prompt).strip(", ")

    strength = float(request.form.get("strength", "0.5"))
    guidance_scale = float(request.form.get("guidance_scale", "0.0"))
    steps = int(request.form.get("steps", "4"))
    seed_str = request.form.get("seed", "")
    width = int(request.form.get("width", "512"))
    height = int(request.form.get("height", "512"))

    num_styles = int(request.form.get("num_styles", "4"))
    num_styles = max(1, min(num_styles, len(STYLE_IMAGES)))  # 1~len 사이로 클램프

    # ---------- 1) Stable Diffusion ----------
    img = img.rotate(90)
    # img.show()
    sd_b64 = None
    if use_sd:
        try:
            generator = None
            if seed_str != "":
                seed = int(seed_str)
                generator = np.random.RandomState(seed)

            # 기존 /infer에서 쓰던 로직과 최대한 비슷하게
            # (입력 회전이 필요하면 img.rotate(90) 등으로 맞춰줘)
            out = pipeline(
                image=img,
                prompt=prompt,
                strength=strength,
                guidance_scale=guidance_scale,
                width=width,
                height=height,
                num_inference_steps=steps,
                generator=generator,
            )
            sd_img: Image.Image = out.images[0]
            sd_img = sd_img.rotate(-90)  # 기존 코드에서 하던 회전이 있다면 유지
            sd_b64 = pil_to_base64(sd_img)

            content_img = sd_img
        except Exception as e:
            return jsonify({"error": f"SD error: {str(e)}"}), 500
    else:
        content_img = img.rotate(-90)
    # content_img.show()

    # ---------- 2) AdaIN Style Transfer N장 ----------
    style_results = []
    try:
        for idx in range(num_styles):
            st_img = run_adain(content_img, STYLE_IMAGES[idx])
            style_results.append({
                "id": str(idx),
                "image": pil_to_base64(st_img),
            })
    except Exception as e:
        return jsonify({"error": f"AdaIN error: {str(e)}"}), 500

    return jsonify({
        "sd": sd_b64,          # 없으면 null
        "styles": style_results
    })


In [ ]:
app.run(host="0.0.0.0", port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.16.119.217:5000
Press CTRL+C to quit
127.0.0.1 - - [04/Dec/2025 16:47:02] "POST /infer HTTP/1.1" 200 -
